## 连接neo4j

In [1]:
from neo4j import GraphDatabase

In [2]:
driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "******"), encrypted=False)

## 添加

In [10]:
# 创建节点
def create_node(tx, name, skill):
    tx.run("CREATE (n:餐饮 {name: $name, skill: $skill}) ", name=name, skill=skill)
with driver.session() as session:
    session.write_transaction(create_node, "巴蜀", "火锅")

In [11]:
# 创建关系
def create_relation(tx, name1, name2):
    tx.run(" match (a:餐饮), (b:餐饮) where a.name = $name1 and b.name = $name2  create (a)-[r:属于]->(b); ", name1=name1, name2=name2)
with driver.session() as session:
    session.write_transaction(create_relation, "巴蜀", "火锅")

In [6]:
# 创建节点时添加关系
def create_node_relation(tx, name1, name2):
    tx.run("create (n:自然{name:$name1}), (m:自然{name:$name2})  create (n) - [r:拥抱] -> (m)", name1=name1, name2=name2)
with driver.session() as session:
    session.write_transaction(create_node_relation, "草地", "太阳")

## 查询

In [7]:
# 查询节点(限制条件)
def query_node_by_limit(tx, name):
    result = tx.run("match (e:华山) where e.name = $name  return e", name=name)
#     print(result.single()[0].values())
    for i in result:
        print(i["e"])
with driver.session() as session:
    session.write_transaction(query_node_by_limit, "令狐冲")

<Node id=0 labels={'华山'} properties={'name': '令狐冲', 'skill': '易筋经'}>


In [8]:
# 查询节点
def query_node(tx):
    result = tx.run("match (e:华山) return e")
    for i in result:
        print(i["e"])
with driver.session() as session:
    session.write_transaction(query_node)

<Node id=0 labels={'华山'} properties={'name': '令狐冲', 'skill': '易筋经'}>
<Node id=1 labels={'华山'} properties={'name': '岳不群', 'skill': '易筋经'}>


In [9]:
# 根据关系查询节点
def query_node_by_relation(tx, name1, name2):
    result = tx.run("match R = (p1:华山) - [r:师父] ->(p2:华山) where p1.name=$name1 and p2.name=$name2 return R;", 
                    name1=name1, name2=name2)
    for i in result:
        print(i["R"])
with driver.session() as session:
    session.write_transaction(query_node_by_relation, "令狐冲", "岳不群")

<Path start=<Node id=0 labels={'华山'} properties={'name': '令狐冲', 'skill': '易筋经'}> end=<Node id=1 labels={'华山'} properties={'name': '岳不群', 'skill': '易筋经'}> size=1>


## 更新

In [10]:
# 更新关系
def update_relation(tx, name1, name2):
    tx.run("MATCH (n:华山 {name:$name1})-[r:朋友]->(m:华山 {name:$name2}) CREATE (n)-[r2:师父]->(m) SET r2 = r WITH r DELETE r", name1=name1, name2=name2)
with driver.session() as session:
    session.write_transaction(update_relation, "令狐冲", "岳不群")

In [11]:
# 更新属性
def update_attr(tx, name, skill):
    message = tx.run("match (e:华山 {name: $name}) set e.skill = $skill", name=name, skill=skill)
    print(message)
with driver.session() as session:
    session.write_transaction(update_attr, "令狐冲", "吸星大法")

In [12]:
# 更新节点
def update_node(tx, name, new_name):
    tx.run("match (e:华山 {name: $name}) set e.name = $new_name", name=name, new_name=new_name)
with driver.session() as session:
    session.write_transaction(update_node, "令狐冲", "LHC")

## 删除

In [13]:
# 删除属性
def del_attr(tx, name):
    message = tx.run("match (n:华山 {name:$name}) remove n.skill", name=name)
    print(message)
with driver.session() as session:
    session.write_transaction(del_attr, "LHC")

In [14]:
# 删除关系
def del_relation(tx, name1, name2):
    message = tx.run(" match (n:华山 {name:$name1}) - [r:师父] -> (m:华山 {name:$name2}) delete r", name1=name1, name2=name2)
with driver.session() as session:
    session.write_transaction(del_relation, "LHC", "岳不群")

In [15]:
# 删除节点
def del_node(tx, name):
    message = tx.run("match (n:华山 {name:$name}) delete n", name=name)
with driver.session() as session:
    session.write_transaction(del_node, "LHC")

In [16]:
# 删除标签
def del_label(tx):
    message = tx.run("match (n:华山) delete n")
with driver.session() as session:
    session.write_transaction(del_label)

## 封装为类

In [ ]:
from neo4j import GraphDatabase

class HelloWorldExample(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password), encrypted=False)

    def close(self):
        self._driver.close()

    def print_greeting(self, message):
        with self._driver.session() as session:
            greeting = session.write_transaction(self._create_and_return_greeting, message)
            print(greeting)

    @staticmethod
    def _create_and_return_greeting(tx, message):
        result = tx.run("CREATE (a:Greeting) "
                        "SET a.message = $message "
                        "RETURN a.message + ', from node ' + id(a)", message=message)
        return result.single()[0]
    
example = HelloWorldExample("bolt://localhost:7687", "neo4j", "******")